In [ ]:
model_dir='./munshi-ai'

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# === Load the fine-tuned model ===
model = AutoModelForCausalLM.from_pretrained(
    model_dir,  # path to your fine-tuned model
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
    cache_dir="/tmp/phi4_cache"
)
tokenizer = AutoTokenizer.from_pretrained(model_dir)
print('model sucessfully loaded ')

In [ ]:
import re
import json

# === Generate predictions ===
for test in test_cases:
    col_str = ", ".join(test["columns"])
    prompt = (
        f"<|system|>\nYou are an expert Python assistant. Generate valid Pandas code based on the user's query.\n"
        f"The DataFrame contains the following columns: {col_str}\n<|end|>\n"
        f"<|user|>\n{test['query']}\n<|end|>\n"
        "<|assistant|>\n"
    )

    print("\n🧪 Prompt:")
    print(prompt)
    
    output = generator(
        prompt,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=50
    )

    full_text = output[0]["generated_text"]
    # print("\n📤 Raw Output:")
    # print(full_text)

    # === Extract JSON object from text
    json_match = re.search(r"\{.*?\}", full_text, re.DOTALL)
    if json_match:
        try:
            raw_json = json_match.group(0).replace("<|end|>", "")
            result = json.loads(raw_json)
            df_code = result.get("df_code", None)
            print("✅ Extracted df_code:", df_code)
        except Exception as e:
            print("❌ JSON parse error:", e)
    else:
        print("❌ No JSON block found.")
